In [1]:
import pandas as pd
from API_Key import key, username, password
import psycopg2 as pg2
import json
from datetime import datetime

In [2]:
conn = pg2.connect(database='TFT', user=username, password=password)
#c = conn.cursor()
df = pd.read_sql(sql=("select * from matches"),con=conn)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7928 entries, 0 to 7927
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   match            7928 non-null   object 
 1   game_datetime    7928 non-null   float64
 2   game_length      7928 non-null   float64
 3   game_version     7928 non-null   object 
 4   gold_left        7928 non-null   int64  
 5   last_round       7928 non-null   int64  
 6   level            7928 non-null   int64  
 7   placement        7928 non-null   int64  
 8   puuid            7928 non-null   object 
 9   time_eliminated  7928 non-null   float64
 10  total_damage     7928 non-null   int64  
 11  traits           7928 non-null   object 
 12  units            7928 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 805.3+ KB


In [4]:
df.head()

match  game_datetime  game_length  \
0  NA1_4107203113   1.637190e+12  2028.263306   
1  NA1_4107203113   1.637190e+12  2028.263306   
2  NA1_4107203113   1.637190e+12  2028.263306   
3  NA1_4107203113   1.637190e+12  2028.263306   
4  NA1_4107203113   1.637190e+12  2028.263306   

                                        game_version  gold_left  last_round  \
0  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          34   
1  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          27   
2  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          1          33   
3  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          1          30   
4  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          27   

   level  placement                                              puuid  \
0      8          3  2-xYFcIJUem__n9qL8bVWDc7uuuvvnp-SuB4VqKmGiPBFw...   
1      7          7  rIB0yQoA9jT06yokBJjP0wQQRbXe2xwZK1dlIrFwi471Im...   
2      8          5  WiQACHT1c3gcMNkdAVHuRybWz48Dsbu3PbTZGlr28dOcCV...   
3      8          6  FqFDLL5R9eVnnGE5TVK-kd1LiZ8GXeS7Ejy73BSnjddxz5...   
4      8          8  4uM1ZdIf5gv4G-I9ZvP9zKajQsJurn24wQQP-UivRpLnh4...   

   time_eliminated  total_damage  \
0      1896.383179            77   
1      1550.396729            37   
2      1842.735352            91   
3      1695.539795            41   
4      1546.322754            53   

                                              traits  \
0  [{"name": "Set6_Bruiser", "num_units": 2, "sty...   
1  [{"name": "Set6_Academy", "num_units": 2, "sty...   
2  [{"name": "Set6_Academy", "num_units": 1, "sty...   
3  [{"name": "Set6_Academy", "num_units": 1, "sty...   
4  [{"name": "Set6_Academy", "num_units": 1, "sty...   

                                               units  
0  [{"character_id": "TFT6_Vi", "items": [], "nam...  
1  [{"character_id": "TFT6_Graves", "items": [], ...  
2  [{"character_id": "TFT6_KogMaw", "items": [12,...  
3  [{"character_id": "TFT6_Swain", "items": [2193...  
4  [{"character_id": "TFT6_Darius", "items": [], ...

In [5]:
df['game_datetime'] = df['game_datetime'].apply(
                                            lambda x: datetime.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))

In [6]:
df = df[df['game_datetime'] >= "2021-11-04"]

In [7]:
# Getting champions from Latest Set
from bs4 import BeautifulSoup
import requests
url = 'https://app.mobalytics.gg/tft/champions'
soup = BeautifulSoup(requests.get(url).content, 'html.parser')

champs = []
for champ in soup.find("div", class_='m-1o47yso').find_all('a'):
    champs.append(champ['href'])

# removing excess string in champion name
for position, champ in enumerate(champs):
    champs[position] = champ.replace("/tft/champions/", "")

In [8]:
# creating new columns for each champ in the TFT set and filling values with 0
for champ in champs:
    df[champ] = 0

In [9]:
df.head()

match        game_datetime  game_length  \
0  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
1  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
2  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
3  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
4  NA1_4107203113  2021-11-17 23:03:32  2028.263306   

                                        game_version  gold_left  last_round  \
0  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          34   
1  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          27   
2  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          1          33   
3  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          1          30   
4  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          27   

   level  placement                                              puuid  \
0      8          3  2-xYFcIJUem__n9qL8bVWDc7uuuvvnp-SuB4VqKmGiPBFw...   
1      7          7  rIB0yQoA9jT06yokBJjP0wQQRbXe2xwZK1dlIrFwi471Im...   
2      8          5  WiQACHT1c3gcMNkdAVHuRybWz48Dsbu3PbTZGlr28dOcCV...   
3      8          6  FqFDLL5R9eVnnGE5TVK-kd1LiZ8GXeS7Ejy73BSnjddxz5...   
4      8          8  4uM1ZdIf5gv4G-I9ZvP9zKajQsJurn24wQQP-UivRpLnh4...   

   time_eliminated  ...  vex vi viktor  warwick  yone  yuumi  zac  ziggs  \
0      1896.383179  ...    0  0      0        0     0      0    0      0   
1      1550.396729  ...    0  0      0        0     0      0    0      0   
2      1842.735352  ...    0  0      0        0     0      0    0      0   
3      1695.539795  ...    0  0      0        0     0      0    0      0   
4      1546.322754  ...    0  0      0        0     0      0    0      0   

   zilean  zyra  
0       0     0  
1       0     0  
2       0     0  
3       0     0  
4       0     0  

[5 rows x 72 columns]

In [10]:
df['units']

0       [{"character_id": "TFT6_Vi", "items": [], "nam...
1       [{"character_id": "TFT6_Graves", "items": [], ...
2       [{"character_id": "TFT6_KogMaw", "items": [12,...
3       [{"character_id": "TFT6_Swain", "items": [2193...
4       [{"character_id": "TFT6_Darius", "items": [], ...
                              ...                        
7923    [{"character_id": "TFT6_Caitlyn", "items": [],...
7924    [{"character_id": "TFT6_Poppy", "items": [], "...
7925    [{"character_id": "TFT6_Kassadin", "items": []...
7926    [{"character_id": "TFT6_Graves", "items": [], ...
7927    [{"character_id": "TFT6_Katarina", "items": []...
Name: units, Length: 6552, dtype: object

In [11]:
df['traits'] = df['traits'].apply(lambda x: json.loads(x))
df['units'] = df['units'].apply(lambda x: json.loads(x))


In [12]:
active_traits = []
for match in df.traits:
    active_traits.append([{trait['name']:trait['tier_current']} 
                          for trait in match 
                          if trait['tier_current'] >=1])

In [13]:
active_traits

[[{'Set6_Bruiser': 1},
  {'Set6_Chemtech': 1},
  {'Set6_Enchanter': 2},
  {'Set6_Scholar': 1},
  {'Set6_Scrap': 1},
  {'Set6_Sister': 1},
  {'Set6_Socialite': 1},
  {'Set6_Twinshot': 1}],
 [{'Set6_Academy': 1},
  {'Set6_Enforcer': 1},
  {'Set6_Mutant': 1},
  {'Set6_Protector': 3},
  {'Set6_Sniper': 1},
  {'Set6_Twinshot': 1}],
 [{'Set6_Arcanist': 1},
  {'Set6_Bruiser': 2},
  {'Set6_Chemtech': 1},
  {'Set6_Cuddly': 1},
  {'Set6_Glutton': 1},
  {'Set6_Mutant': 2},
  {'Set6_Scholar': 1}],
 [{'Set6_Colossus': 1},
  {'Set6_Cuddly': 1},
  {'Set6_Imperial': 1},
  {'Set6_Socialite': 3}],
 [{'Set6_Assassin': 2},
  {'Set6_Bodyguard': 1},
  {'Set6_Cuddly': 1},
  {'Set6_Syndicate': 2}],
 [{'Set6_Bodyguard': 1},
  {'Set6_Clockwork': 2},
  {'Set6_Enchanter': 2},
  {'Set6_Sniper': 1},
  {'Set6_Socialite': 1}],
 [{'Set6_Academy': 4}, {'Set6_Arcanist': 1}, {'Set6_Cuddly': 1}],
 [{'Set6_Academy': 2},
  {'Set6_Bodyguard': 1},
  {'Set6_Challenger': 2},
  {'Set6_Cuddly': 1},
  {'Set6_Enforcer': 1},
  {'Set

In [14]:
df['active_traits'] = active_traits

In [15]:
items = list()
carry = list()
for row in df['units']:
    for resources in row:
        if len(resources['items']) >=1:
            items.append(resources['items'])
            carry.append(resources['name'])

In [42]:
army = []
for unit in df['units']:
    champs = []
    for champ in unit:
        champs.append(champ['character_id'][5:])
    army.append(champs) 

In [43]:
army

[['Vi', 'Lissandra', 'Taric', 'DrMundo', 'Orianna', 'Janna', 'Urgot', 'Jinx'],
 ['Graves', 'Caitlyn', 'Kassadin', 'Garen', 'Blitzcrank', 'KogMaw', 'DrMundo'],
 ['KogMaw',
  'Lissandra',
  'Malzahar',
  'ChoGath',
  'DrMundo',
  'Yuumi',
  'TahmKench'],
 ['Swain', 'Talon', 'Seraphine', 'Sion', 'Yuumi', 'Galio'],
 ['Darius',
  'TwistedFate',
  'Shaco',
  'Ekko',
  'Braum',
  'DrMundo',
  'Yuumi',
  'Akali'],
 ['Camille',
  'Leona',
  'MissFortune',
  'Taric',
  'Braum',
  'Jhin',
  'Orianna',
  'Janna'],
 ['Graves', 'Katarina', 'Leona', 'Vex', 'Lux', 'Yone', 'Yuumi', 'Viktor'],
 ['Leona', 'Braum', 'Lux', 'Yone', 'Fiora', 'Kaisa', 'Yuumi', 'Jayce'],
 ['Poppy', 'Ziggs', 'Tristana', 'Lulu', 'Blitzcrank', 'Vex', 'Heimerdinger'],
 ['Darius',
  'TwistedFate',
  'Katarina',
  'Zyra',
  'Talon',
  'Shaco',
  'Braum',
  'Akali'],
 ['Singed',
  'Zilean',
  'Heimerdinger',
  'Taric',
  'Seraphine',
  'Orianna',
  'Janna',
  'Jayce'],
 ['Kassadin', 'Malzahar', 'ChoGath', 'Sion', 'DrMundo', 'Fiora', 

In [44]:
df['army'] = army

In [52]:
df.columns

Index(['match', 'game_datetime', 'game_length', 'game_version', 'gold_left',
       'last_round', 'level', 'placement', 'puuid', 'time_eliminated',
       'total_damage', 'traits', 'units', 'akali', 'blitz', 'braum', 'caitlyn',
       'camille', 'chogath', 'darius', 'drmundo', 'ekko', 'ezreal', 'fiora',
       'galio', 'gangplank', 'garen', 'graves', 'heimerdinger', 'illaoi',
       'janna', 'jayce', 'jhin', 'jinx', 'kaisa', 'kassadin', 'katarina',
       'kogmaw', 'leona', 'lissandra', 'lulu', 'lux', 'malzahar',
       'miss-fortune', 'orianna', 'poppy', 'quinn', 'samira', 'seraphine',
       'shaco', 'singed', 'sion', 'swain', 'tahm-kench', 'talon', 'taric',
       'tristana', 'trundle', 'twisted-fate', 'twitch', 'urgot', 'veigar',
       'vex', 'vi', 'viktor', 'warwick', 'yone', 'yuumi', 'zac', 'ziggs',
       'zilean', 'zyra', 'active_traits', 'army'],
      dtype='object')

In [63]:
df.rename(columns={"dr-mundo": "drmundo", 
                   "blitz": "blitzcrank",
                  "tahm-kench": "tahmkench",
                  "twisted-fate": "twistedfate",
                  "miss-fortune": "missfortune"}, inplace=True)
df['rainingdummy']=0

In [64]:
# need to fix this to account for unit count per row
for unit in df['army']:
    for champion in unit:
        df[champion.lower()] += 1

In [65]:
df['active_traits']

0       [{'Set6_Bruiser': 1}, {'Set6_Chemtech': 1}, {'...
1       [{'Set6_Academy': 1}, {'Set6_Enforcer': 1}, {'...
2       [{'Set6_Arcanist': 1}, {'Set6_Bruiser': 2}, {'...
3       [{'Set6_Colossus': 1}, {'Set6_Cuddly': 1}, {'S...
4       [{'Set6_Assassin': 2}, {'Set6_Bodyguard': 1}, ...
                              ...                        
7923    [{'Set6_Clockwork': 1}, {'Set6_Cuddly': 1}, {'...
7924    [{'Set6_Arcanist': 1}, {'Set6_Enchanter': 2}, ...
7925    [{'Set6_Arcanist': 1}, {'Set6_Bruiser': 2}, {'...
7926    [{'Set6_Academy': 4}, {'Set6_Challenger': 1}, ...
7927    [{'Set6_Assassin': 3}, {'Set6_Enchanter': 1}, ...
Name: active_traits, Length: 6552, dtype: object

In [66]:
df.head()

match        game_datetime  game_length  \
0  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
1  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
2  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
3  NA1_4107203113  2021-11-17 23:03:32  2028.263306   
4  NA1_4107203113  2021-11-17 23:03:32  2028.263306   

                                        game_version  gold_left  last_round  \
0  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          34   
1  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          27   
2  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          1          33   
3  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          1          30   
4  Version 11.23.409.0111 (Nov 12 2021/12:01:09) ...          0          27   

   level  placement                                              puuid  \
0      8          3  2-xYFcIJUem__n9qL8bVWDc7uuuvvnp-SuB4VqKmGiPBFw...   
1      7          7  rIB0yQoA9jT06yokBJjP0wQQRbXe2xwZK1dlIrFwi471Im...   
2      8          5  WiQACHT1c3gcMNkdAVHuRybWz48Dsbu3PbTZGlr28dOcCV...   
3      8          6  FqFDLL5R9eVnnGE5TVK-kd1LiZ8GXeS7Ejy73BSnjddxz5...   
4      8          8  4uM1ZdIf5gv4G-I9ZvP9zKajQsJurn24wQQP-UivRpLnh4...   

   time_eliminated  ...  yone yuumi   zac  ziggs  zilean  zyra  \
0      1896.383179  ...  1155  1746  1414    984     501   402   
1      1550.396729  ...  1155  1746  1414    984     501   402   
2      1842.735352  ...  1155  1746  1414    984     501   402   
3      1695.539795  ...  1155  1746  1414    984     501   402   
4      1546.322754  ...  1155  1746  1414    984     501   402   

                                       active_traits  \
0  [{'Set6_Bruiser': 1}, {'Set6_Chemtech': 1}, {'...   
1  [{'Set6_Academy': 1}, {'Set6_Enforcer': 1}, {'...   
2  [{'Set6_Arcanist': 1}, {'Set6_Bruiser': 2}, {'...   
3  [{'Set6_Colossus': 1}, {'Set6_Cuddly': 1}, {'S...   
4  [{'Set6_Assassin': 2}, {'Set6_Bodyguard': 1}, ...   

                                                army  raining_dummy  \
0  [Vi, Lissandra, Taric, DrMundo, Orianna, Janna...              0   
1  [Graves, Caitlyn, Kassadin, Garen, Blitzcrank,...              0   
2  [KogMaw, Lissandra, Malzahar, ChoGath, DrMundo...              0   
3      [Swain, Talon, Seraphine, Sion, Yuumi, Galio]              0   
4  [Darius, TwistedFate, Shaco, Ekko, Braum, DrMu...              0   

   rainingdummy  
0           888  
1           888  
2           888  
3           888  
4           888  

[5 rows x 76 columns]